In [6]:
class SshConfig 
{
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

class MysqlConfig {
    public $username;
    public $password;
    public $hostname;
    public $port;
    
    public function __construct(
        string $username, string $password, string $hostname, int $port
    )
    {
        $this->username = $username;
        $this->password = $password;
        $this->hostname = $hostname;
        $this->port = $port;
    }
}

PHP Fatal error:  Class named SshConfig already exists in eval()'d code on line 1

In [2]:
$local_port = "3311";
// $tunnel = sprintf(
//     "nohup python -m sshtunnel -U saxenap -P %s -L :%s -R 127.0.0.1:3306 -p 22 db2.nanohub.org",
//     getenv('tunnel_ssh_password'),
//     $local_port
// );
$tunnel = sprintf(
    "sshpass -p '%s' ssh -vfN -o 'StrictHostKeyChecking=no' -L %s:%s:%d %s ", 
    getenv('tunnel_ssh_password'), 
    $local_port,
    getenv('tunnel_remote_bind_address'), 
    getenv('tunnel_remote_bind_port'),
    getenv('tunnel_ssh_host')
);
$ssh_result = shell_exec($tunnel);
print_r($ssh_result);

// $connection = ssh2_connect(getenv('tunnel_ssh_host'), getenv('tunnel_ssh_port'));
// ssh2_auth_password($connection, getenv('tunnel_ssh_username'), getenv('tunnel_ssh_password'));
// ssh2_tunnel($connection, '127.0.0.1', 3306);



mysqli_report(MYSQLI_REPORT_ERROR | MYSQLI_REPORT_STRICT);
$db_connection = mysqli_connect(
    '127.0.0.1', 
    'saxenap_ro',
    getenv('db_password'), 
    'nanohub',
    $local_port
);

// $dsn = sprintf('mysql:host=%s;port=%s;dbname=nanohub', getenv('db_host'), $local_port);
// $db_connection = new PDO($dsn, 'saxenap_ro', getenv('db_password'));


$query = "SELECT registerDate, username FROM nanohub.jos_users LIMIT 10;";
$result = $db_connection->query($query);
while ($row = $result->fetch_row()) {
    print_r($row);
}

Array
(
    [0] => 2014-11-13 21:09:09
    [1] => hubrepo
)



Array
(
    [0] => 
    [1] => admin
)



Array
(
    [0] => 2008-11-19 22:51:04
    [1] => support
)



Array
(
    [0] => 2008-11-18 17:29:56
    [1] => gridstat
)



Array
(
    [0] => 2008-11-11 19:17:04
    [1] => ncn
)



Array
(
    [0] => 2014-06-26 19:38:57
    [1] => nanohub
)



Array
(
    [0] => 2007-11-13 20:22:04
    [1] => repo
)



Array
(
    [0] => 2007-08-30 15:17:04
    [1] => violin
)



Array
(
    [0] => 2000-01-11 17:10:00
    [1] => mireles
)



Array
(
    [0] => 2000-07-05 07:07:09
    [1] => k_dadesh
)



In [2]:
class SshTunneledDb2Connection 
{
    public $ssh_config;
    public $mysql_config;
    
    public function __construct(SshConfig $ssh_config, MysqlConfig $mysql_config)
    {
        $this->ssh_config = $ssh_config;
        $this->mysql_config = $mysql_config;
    }
    
    public function query(string $query)
    {    
        
        // $ssh_string = "ssh -f -L 3307:127.0.0.1:3306 ";
        // $ssh_string .= $this->ssh_config->username . "@" . $this->ssh_config->hostname;

        // $db = mysqli_connect('127.0.0.1', ‘sqluser’, ‘sqlpassword’, ‘rjmadmin’, 3307);
        
        $connection = ssh2_connect($this->ssh_config->hostname, $this->ssh_config->port);
        if($connection === FALSE){
            throw new RuntimeException("Failed to connect with SSH server.");
        }

        if(!ssh2_auth_password($connection, $this->ssh_config->username, $this->ssh_config->password)){
              throw new RuntimeException("Failed to authenticate to SSH server");
        }
        
        $ssh_query ='ssh -L 3307:' . $this->ssh_config->hostname.':' .$this->mysql_config->port;
        $ssh_query .= ' ' . $query;
    
        $response = ssh2_exec($connection, $ssh_query);
        
        $error = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
        $error_response = stream_get_contents($error);

        stream_set_blocking($response, true);
        stream_set_blocking($error, true);
        
        return stream_get_contents($response);
    }
}

class SshTunneledConnectionFactory
{
    static public function create_from_environment()
    {
        $ssh_config = new SshConfig(
            getenv('tunnel_ssh_username'),
            getenv('tunnel_ssh_password'),
            getenv('tunnel_ssh_host'),
            getenv('tunnel_ssh_port')
        );
        
        $mysql_config = new MysqlConfig(
            getenv('db_user'),
            getenv('db_password'),
            getenv('db_host'),
            getenv('tunnel_remote_bind_port')
        );
        
        return new SshTunneledDb2Connection($ssh_config, $mysql_config);
    }
}

$connection = SshTunneledConnectionFactory::create_from_environment();

$query = sprintf(
    "php -r mysqli_connect(%s, %s, %s, %s, %d); SELECT id, username FROM nanohub.jos_users LIMIT 10;", 
    getenv('db_host'), 
    getenv('db_user'),
    getenv('db_password'), 
    'nanohub',
    getenv('tunnel_remote_bind_port')
);
print_r($connection->query($query));

true

In [3]:
// $ssh['username'] = getenv('tunnel_ssh_username');
// $ssh['password'] = getenv('tunnel_ssh_password');
// $ssh['hostname'] = getenv('tunnel_ssh_host');
// $ssh['port'] = getenv('tunnel_ssh_port');

// $mysql['username'] = getenv('db_user');
// $mysql['password'] = getenv('db_password');
// $mysql['hostname'] = getenv('db_host');
// $mysql['port'] = getenv('tunnel_remote_bind_port');

In [4]:
$query = 'SELECT * FROM nanohub.jos_faq';

$ssh_query ='ssh -L 3307:' . getenv('tunnel_ssh_host').':' . getenv('tunnel_remote_bind_port').'; 
echo "' . str_replace( '"', '\'', stripslashes( $query ) ) . '" | mysql -u '.getenv('db_user').' -h '.getenv('db_host').' --password='.getenv('db_password');

$response = ssh2_exec($connection, $ssh_query);
$error_response = ssh2_fetch_stream($response, SSH2_STREAM_STDERR);
print_r(stream_get_contents($error_response));

stream_set_blocking($response, true);
stream_set_blocking($error_response, true);

$results = stream_get_contents($response);
if (is_null($results ))
    echo "Empty response/result";

else 
    print_r($results);

ssh2_disconnect ( $connection );

PHP Warning:  ssh2_exec() expects parameter 1 to be resource, object given in /home/saxenap/.jupyter-php/pkgs/vendor/litipk/jupyter-php/src/Actions/ExecuteAction.php(115) : eval()'d code on line 4